In [1]:
from matplotlib.pyplot import legend

import character
import simulator

raw_data = simulator.simulate_combat(
    character.generate_archers(),
    target_ac=13,
    iterations=100
)

In [2]:
import pandas as pd

df = pd.DataFrame(raw_data)
display(df)

,iteration_number,round_number,name,level,round_damage
0,0,1,Ranger Archery,1,8
1,0,1,Ranger TwoWeaponFighting,1,8
2,0,1,Ranger Archery,2,8
3,0,1,Ranger TwoWeaponFighting,2,0
4,0,1,Ranger Archery,3,12
...,...,...,...,...,...
11995,99,5,Ranger TwoWeaponFighting,10,28
11996,99,5,Ranger Archery,11,26
11997,99,5,Ranger TwoWeaponFighting,11,29
11998,99,5,Ranger Archery,12,27


In [3]:
df_per_round_stats = df.groupby(['iteration_number', 'level', 'name']).agg(
    # mean_round_damage=('round_damage', 'mean'),
    median_round_damage=('round_damage', 'median'),
    # max_round_damage=('round_damage', 'max'),
    percentile_10_round_damage=('round_damage', lambda x: x.quantile(0.1)),
    percentile_90_round_damage=('round_damage', lambda x: x.quantile(0.9))
).reset_index()

df_per_round_stats


,iteration_number,level,name,median_round_damage,percentile_10_round_damage,percentile_90_round_damage
0,0,1,Ranger Archery,6.0,2.0,10.4
1,0,1,Ranger TwoWeaponFighting,13.0,8.4,18.4
2,0,2,Ranger Archery,11.0,8.0,12.6
3,0,2,Ranger TwoWeaponFighting,14.0,4.0,18.2
4,0,3,Ranger Archery,10.0,10.0,12.0
...,...,...,...,...,...,...
2395,99,10,Ranger TwoWeaponFighting,28.0,21.4,35.0
2396,99,11,Ranger Archery,26.0,23.0,29.0
2397,99,11,Ranger TwoWeaponFighting,29.0,16.2,33.8
2398,99,12,Ranger Archery,25.0,21.6,27.6


In [4]:
df_overall = df_per_round_stats.groupby(['level', 'name']).agg(
    median_DPR=('median_round_damage', 'mean'),
    percentile_10_DPR=('percentile_10_round_damage', 'mean'),
    percentile_90_DPR=('percentile_90_round_damage', 'mean')
).reset_index()

df_overall

,level,name,median_DPR,percentile_10_DPR,percentile_90_DPR
0,1,Ranger Archery,8.26,3.454,12.186
1,1,Ranger TwoWeaponFighting,10.90,5.830,15.736
2,2,Ranger Archery,8.16,3.234,12.062
3,2,Ranger TwoWeaponFighting,10.34,5.592,15.388
4,3,Ranger Archery,8.89,4.292,12.830
5,3,Ranger TwoWeaponFighting,13.28,7.452,18.406
6,4,Ranger Archery,9.86,4.592,12.990
7,4,Ranger TwoWeaponFighting,13.89,8.098,18.104
8,5,Ranger Archery,20.15,14.048,25.070
9,5,Ranger TwoWeaponFighting,20.77,14.442,26.464


In [30]:
import plotly.graph_objects as go
import plotly.express as px


fig = go.Figure()


for name_value in df_overall['name'].unique():
    df_name = df_overall[df_overall['name'] == name_value]
    color = px.colors.qualitative.Set1[df_overall['name'].unique().tolist().index(name_value)]
    
    # fig.add_trace(go.Scatter(
    #     x=df_name['level'],
    #     y=df_name['percentile_90_DPR'],
    #     mode='lines',
    #     name=f'{name_value} DPR 90%',
    #     line=dict(color=color, dash='dash')
    # ))
    # fig.add_trace(go.Scatter(
    #     x=df_name['level'],
    #     y=df_name['percentile_10_DPR'],
    #     mode='lines',
    #     name=f'{name_value} DPR 10%',
    #     line=dict(color=color, dash='dot')
    # ))
    fig.add_trace(go.Scatter(
        x=df_name['level'],
        y=df_name['median_DPR'],
        mode='lines',
        name=f'{name_value} DPR Median',
        line=dict(color=color)
    ))

fig.show()